# Assignment 3 - Building a Custom Visualization

---

In this assignment you must choose one of the options presented below and submit a visual as well as your source code for peer grading. The details of how you solve the assignment are up to you, although your assignment must use matplotlib so that your peers can evaluate your work. The options differ in challenge level, but there are no grades associated with the challenge level you chose. However, your peers will be asked to ensure you at least met a minimum quality for a given technique in order to pass. Implement the technique fully (or exceed it!) and you should be able to earn full grades for the assignment.


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Ferreira, N., Fisher, D., & Konig, A. C. (2014, April). [Sample-oriented task-driven visualizations: allowing users to make better, more confident decisions.](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/Ferreira_Fisher_Sample_Oriented_Tasks.pdf) 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;In Proceedings of the SIGCHI Conference on Human Factors in Computing Systems (pp. 571-580). ACM. ([video](https://www.youtube.com/watch?v=BI7GAs-va-Q))


In this [paper](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/Ferreira_Fisher_Sample_Oriented_Tasks.pdf) the authors describe the challenges users face when trying to make judgements about probabilistic data generated through samples. As an example, they look at a bar chart of four years of data (replicated below in Figure 1). Each year has a y-axis value, which is derived from a sample of a larger dataset. For instance, the first value might be the number votes in a given district or riding for 1992, with the average being around 33,000. On top of this is plotted the 95% confidence interval for the mean (see the boxplot lectures for more information, and the yerr parameter of barcharts).

<br>
<img src="readonly/Assignment3Fig1.png" alt="Figure 1" style="width: 400px;"/>
<h4 style="text-align: center;" markdown="1">  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Figure 1 from (Ferreira et al, 2014).</h4>

<br>

A challenge that users face is that, for a given y-axis value (e.g. 42,000), it is difficult to know which x-axis values are most likely to be representative, because the confidence levels overlap and their distributions are different (the lengths of the confidence interval bars are unequal). One of the solutions the authors propose for this problem (Figure 2c) is to allow users to indicate the y-axis value of interest (e.g. 42,000) and then draw a horizontal line and color bars based on this value. So bars might be colored red if they are definitely above this value (given the confidence interval), blue if they are definitely below this value, or white if they contain this value.


<br>
<img src="readonly/Assignment3Fig2c.png" alt="Figure 1" style="width: 400px;"/>
<h4 style="text-align: center;" markdown="1">  Figure 2c from (Ferreira et al. 2014). Note that the colorbar legend at the bottom as well as the arrows are not required in the assignment descriptions below.</h4>

<br>
<br>

**Easiest option:** Implement the bar coloring as described above - a color scale with only three colors, (e.g. blue, white, and red). Assume the user provides the y axis value of interest as a parameter or variable.


**Harder option:** Implement the bar coloring as described in the paper, where the color of the bar is actually based on the amount of data covered (e.g. a gradient ranging from dark blue for the distribution being certainly below this y-axis, to white if the value is certainly contained, to dark red if the value is certainly not contained as the distribution is above the axis).

**Even Harder option:** Add interactivity to the above, which allows the user to click on the y axis to set the value of interest. The bar colors should change with respect to what value the user has selected.

**Hardest option:** Allow the user to interactively set a range of y values they are interested in, and recolor based on this (e.g. a y-axis band, see the paper for more details).

---

*Note: The data given for this assignment is not the same as the data used in the article and as a result the visualizations may look a little different.*

In [5]:
# Use the following data for this assignment:

import pandas as pd
import numpy as np

np.random.seed(12345)

df = pd.DataFrame([np.random.normal(32000,200000,3650), 
                   np.random.normal(43000,100000,3650), 
                   np.random.normal(43500,140000,3650), 
                   np.random.normal(48000,70000,3650)], 
                  index=[1992,1993,1994,1995])

In [6]:
%matplotlib widget
import matplotlib.pyplot as plt
import scipy.stats as st
import matplotlib.transforms as transforms
from matplotlib import cm, colors
from itertools import chain
from matplotlib.widgets import Slider, Button

In [9]:
# Find means and standard errors for each distribution
means = df.mean(axis=1) 
ses = df.std(axis=1)/np.sqrt(df.shape[1])

# Find deltas between mean and 95% confidence interval for each distribution
def find_interval_size(row):
    low, high = st.norm.interval(alpha=0.95, loc=np.mean(row), scale=st.sem(row))
    return (high - low)/2

interval_sizes = df.apply(find_interval_size, axis=1)

# Set initial y values
y1 = 36000
y2 = 42000

# Helper function to find p-scores for y1 and y2 relative to each sample distribution.
def find_pscores(a, b):
    out = []
    for i in range(4):
        y1_p = st.norm.cdf((a - means.iloc[i])/ ses.iloc[i])
        y2_p = st.norm.cdf((b - means.iloc[i])/ ses.iloc[i])
        out.append([y1_p, y2_p])
    return out

# Define the colormap and helper function for getting the likelihood of sample falling in y range
cmap = cm.get_cmap('Reds')

def find_colors(scores):
    return [cmap(p[1] - p[0]) for p in scores]

# Get the likelihoods of each sample lying in the y range and map it to colors
bar_colors = find_colors(find_pscores(y1, y2))

In [10]:
# Plot figure and make pretty
fig, ax = plt.subplots()
bars = plt.bar(x=[0.5, 1.5, 2.5, 3.5], height=means, 
               color=bar_colors, alpha=1, 
               yerr=interval_sizes, capsize=10, align='center', width=1)

xy_axes = plt.axis([0, 6, 0, 55000])
x_labels = plt.xticks(ticks=[0.5, 1.5, 2.5, 3.5], labels=df.index)
y1_line = plt.axhline(y=y1, xmax=0.8, color='blue', alpha=1, linewidth=1)
y2_line = plt.axhline(y=y2, xmax=0.8, color='green', alpha=1, linewidth=1)

# Put y value text on the plot
trans = transforms.blended_transform_factory(ax.get_yticklabels()[0].get_transform(), 
                                             ax.transData)
y1_text = ax.text(0.92, y1, "{:.0f}".format(y1), color="blue", transform=trans, 
        ha="right", va="center")
y2_text = ax.text(0.92, y2, "{:.0f}".format(y2), color="green", transform=trans, 
        ha="right", va="center")

# Make a colorbar
my_bar = plt.colorbar(cm.ScalarMappable(norm=colors.Normalize(), cmap=cmap), orientation='horizontal', 
             shrink=0.8, pad=0.1, aspect=40)

# Caption the colorbar
caption = 'Color shows likelihood that sample is in the range y1 to y2'
bar_caption = fig.text(.5, .1, caption, ha='center')

# Remove top and left borders
for side in ['top', 'right']:
    ax.spines[side].set_visible(False)


### SLIDERS ###
# I just guessed around with positining. If I were smarter I would have pulled in the exact  
# values from axes objects to line them up
y1_slider_ax  = fig.add_axes([0.85, 0.3025, 0.015, 0.58], facecolor='grey')
y1_slider = Slider(y1_slider_ax, 'y1', 0, 55000, valinit=y1, orientation='vertical', 
                   dragging=True, facecolor='blue', alpha=0.8)
y1_slider.valtext.set_visible(False)

y2_slider_ax = fig.add_axes([0.94, 0.3025, 0.015, 0.58], facecolor='grey')
y2_slider = Slider(y2_slider_ax, 'y2', 0, 55000, valinit=y2, orientation='vertical', 
                   dragging=True, facecolor='green', alpha=0.8)
y2_slider.valtext.set_visible(False)

# Define slider events and link them
def y1_slider_on_changed(val):
    y1_line.set_ydata(val)
    y1_text.set_y(val)
    y1_text.set_text("{:.0f}".format(val))
    recolor = find_colors(find_pscores(val, y2_slider.val))
    for i, c in enumerate(recolor):
        bars[i].set_color(c)
    
def y2_slider_on_changed(val):
    y2_line.set_ydata(val)
    y2_text.set_y(val)
    y2_text.set_text("{:.0f}".format(val))
    recolor = find_colors(find_pscores(y1_slider.val, val))
    for i, c in enumerate(recolor):
        bars[i].set_color(c)
    
y1_slider.on_changed(y1_slider_on_changed)
y2_slider.on_changed(y2_slider_on_changed)


# Create reset button for initial y1, y2 values
reset_button_ax = fig.add_axes([0.85, 0.025, 0.1, 0.04])
reset_button = Button(reset_button_ax, 'Reset', color='#CACBCF', hovercolor='0.975')

def reset_button_on_clicked(mouse_event):
    y1_slider.reset()
    y2_slider.reset()
    
reset_button.on_clicked(reset_button_on_clicked)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0